## SVM + TDIDF

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data = pd.read_csv('/content/drive/MyDrive/dataset_ner_tsv/merged_file.tsv', sep='\t')

In [3]:
data

,text,label
0,COVID-19,S-Disease
1,COVID-19,S-Disease
2,COVID-19,S-Disease
3,malaria,S-Disease
4,HIV,S-Disease
...,...,...
113149,do,O
113150,Estado,O
113151,do,O
113152,Amazonas,O


In [4]:
data.isnull().sum()

text     1
label    0
dtype: int64

In [5]:
data.dropna(inplace=True)

In [7]:
data.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
#data.duplicated().sum()
#data.drop_duplicates(inplace=True)

In [ ]:
data

,text,label
0,COVID-19,S-Disease
1,COVID-19,S-Disease
2,COVID-19,S-Disease
3,malaria,S-Disease
4,HIV,S-Disease
...,...,...
113149,do,O
113150,Estado,O
113151,do,O
113152,Amazonas,O


In [ ]:
print(data['label'].value_counts())

O                          105303
S-Disease                    1182
B-Disease                    1016
B-Organism                    961
I-Organism                    536
S-Chemical_Substance          496
S-Protien                     463
S-Organism                    402
B-Chemical_Substance          351
B-Protien                     302
I-Disease                     297
B-Medication                  229
S-Medication                  171
S-Anatomical_Substances       151
I-Protien                     143
B-Anatomical_Substances       143
I-Chemical_Substance          137
I-Anatomical_Substances       122
I-Medication                  104
S-Gene                        101
E-Disease                      83
E-Protien                      80
E-Organism                     75
S_Anatomical_Substances        72
B-Gene                         60
E-Medication                   46
E-Anatomical_Substances        42
E-Chemical_Substance           41
I-Gene                         27
E-Gene        

In [8]:
X= data['text']
y =data['label']

In [ ]:
X

0         COVID-19
1         COVID-19
2         COVID-19
3          malaria
4              HIV
            ...   
113149          do
113150      Estado
113151          do
113152    Amazonas
113153      FAPEAM
Name: text, Length: 113153, dtype: object

In [ ]:
y

0         S-Disease
1         S-Disease
2         S-Disease
3         S-Disease
4         S-Disease
            ...    
113149            O
113150            O
113151            O
113152            O
113153            O
Name: label, Length: 113153, dtype: object

In [ ]:
print(len(set(X)))

14376


In [ ]:
print(len(set(y)))

30


In [ ]:
from collections import Counter
print(Counter(y))

Counter({'O': 105303, 'S-Disease': 1182, 'B-Disease': 1016, 'B-Organism': 961, 'I-Organism': 536, 'S-Chemical_Substance': 496, 'S-Protien': 463, 'S-Organism': 402, 'B-Chemical_Substance': 351, 'B-Protien': 302, 'I-Disease': 297, 'B-Medication': 229, 'S-Medication': 171, 'S-Anatomical_Substances': 151, 'I-Protien': 143, 'B-Anatomical_Substances': 143, 'I-Chemical_Substance': 137, 'I-Anatomical_Substances': 122, 'I-Medication': 104, 'S-Gene': 101, 'E-Disease': 83, 'E-Protien': 80, 'E-Organism': 75, 'S_Anatomical_Substances': 72, 'B-Gene': 60, 'E-Medication': 46, 'E-Anatomical_Substances': 42, 'E-Chemical_Substance': 41, 'I-Gene': 27, 'E-Gene': 17})


In [ ]:
#sns.countplot(data['label'])

In [ ]:
#minority_classes = ['B-Organism', 'B-Disease', 'S-Chemical_Substance', 'S-Protien', 'S-Disease', 'B-Chemical_Substance', 'B-Protien', 'I-Organism', 'S-Organism', 'I-Disease', 'B-Medication', 'I-Protien', 'I-Chemical_Substance', 'S-Medication', 'B-Anatomical_Substances', 'I-Anatomical_Substances', 'I-Medication', 'S-Gene', 'E-Protien', 'S-Anatomical_Substances', 'E-Organism', 'S_Anatomical_Substances', 'E-Disease', 'B-Gene', 'E-Chemical_Substance', 'E-Medication', 'E-Anatomical_Substances', 'I-Gene', 'E-Gene']
#df_minority = data[data['label'].isin(minority_classes)]
#df_majority = data[~data['label'].isin(minority_classes)]
#from sklearn.utils import resample
#randomly undersample the majority class to match the size of the minority class
#majority_class_size = len(df_minority)
#df_majority_downsampled = resample(df_majority,
                                   #replace=False,      # sample without replacement
                                   #n_samples=majority_class_size,  # match minority class size
                                   #random_state=123)   # set random state for reproducibility
#concatenate the minority class dataframe with the downsampled majority class dataframe
#df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [ ]:
#print(Counter(df_downsampled['label']))

In [ ]:
#import imblearn
#from imblearn.under_sampling import RandomUnderSampler
#rus = RandomUnderSampler(random_state=42)

In [ ]:
#X_rus, y_rus = rus.fit_resample(X, y)

In [ ]:
#X= df_downsampled['text']
#y =df_downsampled['label']

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer to convert texts to vectors
vectorizer = TfidfVectorizer()

In [ ]:
# Convert train and test texts to vectors
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

In [ ]:
# Encoding the classes in numerical values
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [ ]:
unique_elements = set(y_train)
num_unique_elements = len(unique_elements)

In [ ]:
num_unique_elements

30

In [ ]:
# Create SVM classifier
clf = SVC(kernel='rbf', C=1, decision_function_shape='ovr')

In [ ]:
# Train the classifier on the training data
clf.fit(X_train_vectors, y_train)

SVC(C=1)

In [ ]:
y_pred = clf.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9474614466881711


In [ ]:
#testing on new text
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=clf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## LOGISTIC REGRESSION + TDIDF

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create logistic regression classifier
clsf = LogisticRegression(random_state=0, max_iter=1000, multi_class='ovr')

# Train the classifier on the training data
clsf.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = clsf.predict(X_test_vectors)

In [ ]:
y_pred = clsf.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9386681984887986


In [ ]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## RANDOM FOREST + TDIDF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create Random Forest classifier
cl = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier on the training data
cl.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = cl.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9500684901241659


In [ ]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=cl.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## NAIVE BAYES+TFIDF

In [ ]:
# Create Naive Bayes classifier
c = MultinomialNB()

# Train the classifier on the training data
c.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = c.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9305819451195263


In [ ]:
new=""" P. falciparum is the deadliest malaria parasite and the most prevalent on the African continent. P. vivax is the dominant malaria parasite in most countries outside of sub-Saharan Africa."""
n=new.split()
for i in n:
    vector=vectorizer.transform([i])
    pred=cl.predict(vector)
    print(i,encoder.inverse_transform(pred))

P. ['O']
falciparum ['I-Organism']
is ['O']
the ['O']
deadliest ['O']
malaria ['S-Disease']
parasite ['O']
and ['O']
the ['O']
most ['O']
prevalent ['O']
on ['O']
the ['O']
African ['O']
continent. ['O']
P. ['O']
vivax ['B-Organism']
is ['O']
the ['O']
dominant ['O']
malaria ['S-Disease']
parasite ['O']
in ['O']
most ['O']
countries ['O']
outside ['O']
of ['O']
sub-Saharan ['O']
Africa. ['O']


In [ ]:
#confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay
#ConfusionMatrixDisplay.from_predictions(y_test,y_pred)

In [ ]:
# results
#from sklearn.metrics import classification_report
#report = classification_report(y_test,y_pred)
#print(report)

#SVM + COUNT VECTORIZER

In [ ]:
X= data['text']
y =data['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vec = CountVectorizer()
#vectorizer = CountVectorizer(stop_words='english', lowercase=True, max_df=0.8, min_df=5)

# Fit the vectorizer to the training data and transform the training data into a bag-of-words matrix
X_train_vectors = vec.fit_transform(X_train)

# Transform the test data into a bag-of-words matrix using the same vectorizer
X_test_vectors = vec.transform(X_test)

In [ ]:
# Encoding the classes in numerical values
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [ ]:
# Create SVM classifier
clf = SVC(kernel='rbf', C=1, decision_function_shape='ovr')

In [ ]:
clf.fit(X_train_vectors, y_train)

SVC(C=1)

In [ ]:
y_pred = clf.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9486103132870841


In [ ]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## LOGISTIC REGRESSION + COUNT VECTORIZER

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create logistic regression classifier
clsf = LogisticRegression(random_state=0, max_iter=1000, multi_class='ovr')

# Train the classifier on the training data
clsf.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = clsf.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9398170650877116


In [ ]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## RANDOM FOREST + COUNT VECTORIZER

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create Random Forest classifier
cl = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier on the training data
cl.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = cl.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9500684901241659


In [ ]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))

Malaria ['S-Disease']
is ['O']
an ['O']
acute ['O']
febrile ['O']
illness ['O']
caused ['O']
by ['O']
Plasmodium ['B-Organism']
parasites, ['O']
which ['O']
are ['O']
spread ['O']
to ['O']
people ['O']
through ['O']
the ['O']
bites ['O']
of ['O']
infected ['O']
female ['O']
Anopheles ['B-Organism']
mosquitoes. ['O']
It ['O']
is ['O']
preventable ['O']
and ['O']
curable. ['O']


## NAIVE BAYES+COUNT VECTORIZER

In [ ]:
# Create Naive Bayes classifier
c = MultinomialNB()

# Train the classifier on the training data
c.fit(X_train_vectors, y_train)

# Predict the labels of the test data
y_pred = c.predict(X_test_vectors)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9313773143033891


In [ ]:
new="""Malaria is an acute febrile illness caused by Plasmodium parasites, which are spread to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable."""
n=new.split()
for i in n:
    vector=vec.transform([i])
    pred=clsf.predict(vector)
    print(i,encoder.inverse_transform(pred))

## SVM + WORD2VEC

In [86]:
import numpy as np
from gensim.models import Word2Vec

In [87]:
# Split the text and labels into separate lists
sentences = [text.split() for text in data["text"]]
labels = data["label"]

In [88]:
# Train a Word2Vec model on the text data
model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

In [89]:
# Convert each text into a fixed-size vector using the Word2Vec model
X = np.zeros((len(sentences), 100))
for i, sentence in enumerate(sentences):
    for word in sentence:
        if word in model.wv:
            X[i] += model.wv[word]
    X[i] /= len(sentence)

In [90]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [76]:
# Create SVM classifier
clf = SVC(kernel='rbf', C=1, decision_function_shape='ovr')

In [77]:
clf.fit(X_train, y_train)

SVC(C=1)

In [21]:
# Predict the labels of the test data
y_pred = clf.predict(X_test)

In [22]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9460474570279704


In [79]:
# Tokenize a new sentence into a list of words
new_sentence = "Malaria is an infectious disease transmitted by the female Anopheles mosquito and poses a severe threat to human health. At present, antimalarial drugs are the primary treatment for malaria."
new_words = new_sentence.split()

# Use the trained classification model to predict the label of each word in the sentence
for word in new_words:
    # Obtain the vector representation of the word
    if word in model.wv:
        word_vector = model.wv[word]
    else:
        # If the word is not in the vocabulary, use a zero vector instead
        word_vector = np.zeros(model.vector_size)

    # Use the trained classification model to predict the label of the word based on its vector representation
    label = clf.predict([word_vector])
    print("Predicted label for word '{}': {}".format(word, label))

Predicted label for word 'Malaria': ['S-Disease']
Predicted label for word 'is': ['O']
Predicted label for word 'an': ['O']
Predicted label for word 'infectious': ['O']
Predicted label for word 'disease': ['O']
Predicted label for word 'transmitted': ['O']
Predicted label for word 'by': ['O']
Predicted label for word 'the': ['O']
Predicted label for word 'female': ['O']
Predicted label for word 'Anopheles': ['B-Organism']
Predicted label for word 'mosquito': ['O']
Predicted label for word 'and': ['O']
Predicted label for word 'poses': ['O']
Predicted label for word 'a': ['O']
Predicted label for word 'severe': ['O']
Predicted label for word 'threat': ['O']
Predicted label for word 'to': ['O']
Predicted label for word 'human': ['O']
Predicted label for word 'health.': ['O']
Predicted label for word 'At': ['O']
Predicted label for word 'present,': ['O']
Predicted label for word 'antimalarial': ['O']
Predicted label for word 'drugs': ['O']
Predicted label for word 'are': ['O']
Predicted l

## LOGISTIC REGRESSION + WORD2VEC

In [80]:
from sklearn.linear_model import LogisticRegression

# Create logistic regression classifier
clsf = LogisticRegression(random_state=0, max_iter=1000, multi_class='ovr')

# Train the classifier on the training data
clsf.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = clsf.predict(X_test)

In [81]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9312005655958641


In [100]:
# Tokenize a new sentence into a list of words
new_sentence = "Malaria is an infectious disease transmitted by the female Anopheles mosquito and poses a severe threat to human health."
new_words = new_sentence.split()

# Use the trained classification model to predict the label of each word in the sentence
for word in new_words:
    # Obtain the vector representation of the word
    if word in model.wv:
        word_vector = model.wv[word]
    else:
        # If the word is not in the vocabulary, use a zero vector instead
        word_vector = np.zeros(model.vector_size)

    # Use the trained classification model to predict the label of the word based on its vector representation
    label = clsf.predict([word_vector])
    print("Predicted label for word '{}': {}".format(word, label))

Predicted label for word 'Malaria': ['O']
Predicted label for word 'is': ['O']
Predicted label for word 'an': ['O']
Predicted label for word 'infectious': ['O']
Predicted label for word 'disease': ['O']
Predicted label for word 'transmitted': ['O']
Predicted label for word 'by': ['O']
Predicted label for word 'the': ['O']
Predicted label for word 'female': ['O']
Predicted label for word 'Anopheles': ['O']
Predicted label for word 'mosquito': ['O']
Predicted label for word 'and': ['O']
Predicted label for word 'poses': ['O']
Predicted label for word 'a': ['O']
Predicted label for word 'severe': ['O']
Predicted label for word 'threat': ['O']
Predicted label for word 'to': ['O']
Predicted label for word 'human': ['O']
Predicted label for word 'health.': ['O']


## NAIVE BAYES + WORD2VEC

In [64]:
# Create Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
c = GaussianNB()

# Train the classifier on the training data
c.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = c.predict(X_test)

In [65]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.8102160752949494


In [96]:
# Tokenize a new sentence into a list of words
new_sentence = """SARS-CoV-2 infection that led to the COVID-19 pandemic has changed human health and the economy globally. """
new_words = new_sentence.split()

# Use the trained classification model to predict the label of each word in the sentence
for word in new_words:
    # Obtain the vector representation of the word
    if word in model.wv:
        word_vector = model.wv[word]
    else:
        # If the word is not in the vocabulary, use a zero vector instead
        word_vector = np.zeros(model.vector_size)

    # Use the trained classification model to predict the label of the word based on its vector representation
    label = c.predict([word_vector])
    print("Predicted label for word '{}': {}".format(word, label))

Predicted label for word 'SARS-CoV-2': ['O']
Predicted label for word 'infection': ['I-Disease']
Predicted label for word 'that': ['O']
Predicted label for word 'led': ['O']
Predicted label for word 'to': ['O']
Predicted label for word 'the': ['O']
Predicted label for word 'COVID-19': ['O']
Predicted label for word 'pandemic': ['O']
Predicted label for word 'has': ['O']
Predicted label for word 'changed': ['E-Gene']
Predicted label for word 'human': ['O']
Predicted label for word 'health': ['O']
Predicted label for word 'and': ['O']
Predicted label for word 'the': ['O']
Predicted label for word 'economy': ['E-Gene']
Predicted label for word 'globally.': ['E-Gene']


## RANDOM FOREST +WORD2VEC

In [91]:
from sklearn.ensemble import RandomForestClassifier

# Create Random Forest classifier
cl = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier on the training data
cl.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = cl.predict(X_test)

In [84]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.9470637620962397


In [98]:
# Tokenize a new sentence into a list of words
new_sentence = """SARS-CoV-2 infection that led to the COVID-19 pandemic has changed human health and the economy globally."""
new_words = new_sentence.split()

# Use the trained classification model to predict the label of each word in the sentence
for word in new_words:
    # Obtain the vector representation of the word
    if word in model.wv:
        word_vector = model.wv[word]
    else:
        # If the word is not in the vocabulary, use a zero vector instead
        word_vector = np.zeros(model.vector_size)

    # Use the trained classification model to predict the label of the word based on its vector representation
    label = cl.predict([word_vector])
    print("Predicted label for word '{}': {}".format(word, label))

Predicted label for word 'SARS-CoV-2': ['S-Organism']
Predicted label for word 'infection': ['O']
Predicted label for word 'that': ['O']
Predicted label for word 'led': ['O']
Predicted label for word 'to': ['O']
Predicted label for word 'the': ['O']
Predicted label for word 'COVID-19': ['S-Disease']
Predicted label for word 'pandemic': ['O']
Predicted label for word 'has': ['O']
Predicted label for word 'changed': ['O']
Predicted label for word 'human': ['O']
Predicted label for word 'health': ['O']
Predicted label for word 'and': ['O']
Predicted label for word 'the': ['O']
Predicted label for word 'economy': ['O']
Predicted label for word 'malaria': ['S-Disease']
Predicted label for word 'globally.': ['O']


### TfidfVectorizer +CountVectorizer+SVM

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [7]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)

In [8]:
X = data['text']
y=data['label']

In [9]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [10]:
# Apply CountVectorizer and TfidfVectorizer to the train and test data
cv = CountVectorizer()
tfidf = TfidfVectorizer()

X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [11]:
# Concatenate the CountVectorizer and TfidfVectorizer features
X_train_combined = np.concatenate((X_train_cv.toarray(), X_train_tfidf.toarray()), axis=1)
X_test_combined = np.concatenate((X_test_cv.toarray(), X_test_tfidf.toarray()), axis=1)

In [ ]:
# Create SVM classifier
clf = SVC(kernel='rbf', C=1, decision_function_shape='ovr')
clf.fit(X_train_combined, y_train)
# Predict the labels of the test data
y_pred = clf.predict(X_test_combined)

### TFIDF+ word2vec+SVM